In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from scipy.stats import norm

from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.model_selection import bayesian_search_forecaster_multiseries

In [16]:
# Load time series of multiple lengths and exogenous variables
# ==============================================================================
series = pd.read_csv(
    'https://raw.githubusercontent.com/skforecast/skforecast-datasets/main/data/demo_multi_series.csv'
)
exog = pd.read_csv(
    'https://raw.githubusercontent.com/skforecast/skforecast-datasets/main/data/demo_multi_series_exog.csv'
)

series['timestamp'] = pd.to_datetime(series['timestamp'])
exog['timestamp'] = pd.to_datetime(exog['timestamp'])

display(series.head())
print("")
display(exog.head())

,series_id,timestamp,value
0,id_1000,2016-01-01,1012.500694
1,id_1000,2016-01-02,1158.500099
2,id_1000,2016-01-03,983.000099
3,id_1000,2016-01-04,1675.750496
4,id_1000,2016-01-05,1586.250694


,series_id,timestamp,sin_day_of_week,cos_day_of_week,air_temperature,wind_speed
0,id_1000,2016-01-01,-0.433884,-0.900969,6.416639,4.040115
1,id_1000,2016-01-02,-0.974928,-0.222521,6.366474,4.530395
2,id_1000,2016-01-03,-0.781831,0.623490,6.555272,3.273064
3,id_1000,2016-01-04,0.000000,1.000000,6.704778,4.865404
4,id_1000,2016-01-05,0.781831,0.623490,2.392998,5.228913


In [17]:
# Transform series and exog to dictionaries
# ==============================================================================
series_dict = series_long_to_dict(
    data      = series,
    series_id = 'series_id',
    index     = 'timestamp',
    values    = 'value',
    freq      = 'D'
)

exog_dict = exog_long_to_dict(
    data      = exog,
    series_id = 'series_id',
    index     = 'timestamp',
    freq      = 'D'
)

c:\Users\jaesc2\GitHub\skforecast\skforecast\preprocessing\preprocessing.py:425: MissingValuesWarning: Series 'id_1003' is incomplete. NaNs have been introduced after setting the frequency. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(


In [18]:
# Drop some exogenous variables for series 'id_1000' and 'id_1003'
# ==============================================================================
exog_dict['id_1000'] = exog_dict['id_1000'].drop(columns=['air_temperature', 'wind_speed'])
exog_dict['id_1003'] = exog_dict['id_1003'].drop(columns=['cos_day_of_week'])

In [19]:
# Partition data in train and test
# ==============================================================================
end_train = '2016-07-31 23:59:00'

series_dict_train = {k: v.loc[: end_train,] for k, v in series_dict.items()}
exog_dict_train   = {k: v.loc[: end_train,] for k, v in exog_dict.items()}
series_dict_test  = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test    = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [20]:
# Fit forecaster
# ==============================================================================
regressor = LGBMRegressor(random_state=123, verbose=-1, max_depth=5)
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = regressor, 
                 lags               = 14, 
                 window_features    = RollingFeatures(stats=['mean', 'mean'], window_sizes=[7, 14]),
                 encoding           = "ordinal", 
                 dropna_from_series = False
             )

forecaster.fit(series=series_dict_train, exog=exog_dict_train, suppress_warnings=True)
forecaster

============================== 
ForecasterRecursiveMultiSeries 
============================== 
Regressor: LGBMRegressor 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
Window features: ['roll_mean_7', 'roll_mean_14'] 
Window size: 14 
Series encoding: ordinal 
Series names (levels): id_1000, id_1001, id_1002, id_1003, id_1004 
Exogenous included: True 
Exogenous names: 
    sin_day_of_week, cos_day_of_week, air_temperature, wind_speed 
Transformer for series: None 
Transformer for exog: None 
Weight function included: False 
Series weights: None 
Differentiation order: None 
Training range: 
    'id_1000': ['2016-01-01', '2016-07-31'], 'id_1001': ['2016-07-02',
    '2016-07-31'], 'id_1002': ['2016-01-01', '2016-07-01'], 'id_1003':
    ['2016-01-01', '2016-07-31'], 'id_1004': ['2016-05-02', '2016-07-31'] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: 
    {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0,
    'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': 5,
    'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0,
    'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None,
    'random_state': 123, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0,
    'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1} 
fit_kwargs: {} 
Creation date: 2024-12-28 20:23:51 
Last fit date: 2024-12-28 20:23:51 
Skforecast version: 0.15.0 
Python version: 3.11.10 
Forecaster id: None

In [21]:
preds = forecaster.predict(steps=10, exog=exog_dict_test)
preds

c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:1056: MissingExogWarning: {'wind_speed', 'air_temperature'} not present in `exog` for series 'id_1000'. All values will be NaN. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingExogWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:1056: MissingExogWarning: {'cos_day_of_week'} not present in `exog` for series 'id_1003'. All values will be NaN. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingExogWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:562: MissingValuesWarning: `exog` for series 'id_1000' has missing values. Most machine learning models do not allow missing values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py

,level,pred
2016-08-01,id_1000,1453.312971
2016-08-01,id_1001,2849.347882
2016-08-01,id_1003,2706.851726
2016-08-01,id_1004,7496.555367
2016-08-02,id_1000,1440.763196
2016-08-02,id_1001,2947.579536
2016-08-02,id_1003,2310.075968
2016-08-02,id_1004,8685.425990
2016-08-03,id_1000,1410.151437
2016-08-03,id_1001,2875.847691


In [84]:
preds = forecaster.predict_bootstrapping(
    steps=10, exog=exog_dict_test, n_boot=3, suppress_warnings=True
)
preds.head(3)

,level,pred_boot_0,pred_boot_1,pred_boot_2
2016-08-01,id_1000,1173.586189,1484.675570,1418.862097
2016-08-01,id_1001,2738.406500,3184.698632,2119.125183
2016-08-01,id_1003,2901.172020,2577.806548,2465.634521


In [85]:
preds.index

DatetimeIndex(['2016-08-01', '2016-08-01', '2016-08-01', '2016-08-01',
               '2016-08-02', '2016-08-02', '2016-08-02', '2016-08-02',
               '2016-08-03', '2016-08-03', '2016-08-03', '2016-08-03',
               '2016-08-04', '2016-08-04', '2016-08-04', '2016-08-04',
               '2016-08-05', '2016-08-05', '2016-08-05', '2016-08-05',
               '2016-08-06', '2016-08-06', '2016-08-06', '2016-08-06',
               '2016-08-07', '2016-08-07', '2016-08-07', '2016-08-07',
               '2016-08-08', '2016-08-08', '2016-08-08', '2016-08-08',
               '2016-08-09', '2016-08-09', '2016-08-09', '2016-08-09',
               '2016-08-10', '2016-08-10', '2016-08-10', '2016-08-10'],
              dtype='datetime64[ns]', freq=None)

In [86]:
tru_idx = pd.date_range(
    start='2016-08-03', periods=4, freq='D'
)

In [89]:
no_valid_index = preds.index.difference(tru_idx, sort=False)
no_valid_index

DatetimeIndex(['2016-08-01', '2016-08-02', '2016-08-07', '2016-08-08',
               '2016-08-09', '2016-08-10'],
              dtype='datetime64[ns]', freq=None)

In [91]:
preds.loc[no_valid_index, 'level'] == 'id_1000'

2016-08-01     True
2016-08-01    False
2016-08-01    False
2016-08-01    False
2016-08-02     True
2016-08-02    False
2016-08-02    False
2016-08-02    False
2016-08-07     True
2016-08-07    False
2016-08-07    False
2016-08-07    False
2016-08-08     True
2016-08-08    False
2016-08-08    False
2016-08-08    False
2016-08-09     True
2016-08-09    False
2016-08-09    False
2016-08-09    False
2016-08-10     True
2016-08-10    False
2016-08-10    False
2016-08-10    False
Name: level, dtype: bool

In [98]:
%%timeit

mask_level = preds.loc[no_valid_index, 'level'] == 'id_1000'

preds.loc[no_valid_index, :][mask_level]

412 μs ± 2.16 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [99]:
%%timeit

preds.query("level == 'id_1000'").loc[no_valid_index, :]

1.01 ms ± 16.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [100]:
preds.loc[(no_valid_index) & (preds['level'] == 'level_1'), :]

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
preds = forecaster.predict_interval(
    steps=3, exog=exog_dict_test, n_boot=5, suppress_warnings=True
)
preds

,level,pred,lower_bound,upper_bound
2016-08-01,id_1000,1453.312971,1209.130225,1564.005374
2016-08-01,id_1001,2849.347882,2755.277127,3279.002987
2016-08-01,id_1003,2706.851726,2553.588371,2854.562116
2016-08-01,id_1004,7496.555367,7099.337865,7640.095357
2016-08-02,id_1000,1440.763196,1356.484185,1534.631945
2016-08-02,id_1001,2947.579536,2452.491092,3404.677476
2016-08-02,id_1003,2310.075968,1916.419934,2370.788136
2016-08-02,id_1004,8685.425990,8424.687306,8882.647361
2016-08-03,id_1000,1410.151437,1401.003123,1446.808327
2016-08-03,id_1001,2875.847691,2417.308783,3307.598144


In [ ]:
preds = forecaster.predict_quantiles(
    steps=3, exog=exog_dict_test, n_boot=5, suppress_warnings=True
)
preds

,level,q_0.05,q_0.5,q_0.95
2016-08-01,id_1000,1209.130225,1418.862097,1564.005374
2016-08-01,id_1001,2755.277127,2941.044595,3279.002987
2016-08-01,id_1003,2553.588371,2721.819816,2854.562116
2016-08-01,id_1004,7099.337865,7562.004274,7640.095357
2016-08-02,id_1000,1356.484185,1426.963184,1534.631945
2016-08-02,id_1001,2452.491092,3203.828306,3404.677476
2016-08-02,id_1003,1916.419934,2175.260402,2370.788136
2016-08-02,id_1004,8424.687306,8728.791739,8882.647361
2016-08-03,id_1000,1401.003123,1421.263357,1446.808327
2016-08-03,id_1001,2417.308783,2795.301289,3307.598144


In [ ]:
preds = forecaster.predict_dist(
    steps=3, exog=exog_dict_test, n_boot=5, suppress_warnings=True,
    distribution=norm
)
preds

,level,loc,scale
2016-08-01,id_1000,1402.453610,137.786980
2016-08-01,id_1001,3008.027546,204.323833
2016-08-01,id_1003,2711.079834,117.808039
2016-08-01,id_1004,7412.883313,233.442427
2016-08-02,id_1000,1442.229888,70.417452
2016-08-02,id_1001,3009.582727,383.114991
2016-08-02,id_1003,2140.553473,176.807352
2016-08-02,id_1004,8682.156795,179.282271
2016-08-03,id_1000,1423.810787,17.714483
2016-08-03,id_1001,2853.873400,355.271136


In [ ]:

preds.iloc[:, 1:].apply(
    lambda x: norm.fit(x), axis=1, result_type='expand'
)

,0,1
2016-08-01,770.120295,632.333315
2016-08-01,1606.175689,1401.851856
2016-08-01,1414.443937,1296.635897
2016-08-01,3823.162870,3589.720443
2016-08-02,756.323670,685.906218
2016-08-02,1696.348859,1313.233868
2016-08-02,1158.680412,981.873060
2016-08-02,4430.719533,4251.437262
2016-08-03,720.762635,703.048152
2016-08-03,1604.572268,1249.301132


In [63]:
interval = np.array([5, 95]) / 100

preds[['lower_bound', 'upper_bound']] = (
    preds.iloc[:, 1:].quantile(q=interval, axis=1).transpose()
)
preds = preds[['level', 'lower_bound', 'upper_bound']]
preds

,level,lower_bound,upper_bound
2016-08-01,id_1000,1209.130225,1564.005374
2016-08-01,id_1001,2755.277127,3279.002987
2016-08-01,id_1003,2553.588371,2854.562116
2016-08-01,id_1004,7099.337865,7640.095357
2016-08-02,id_1000,1356.484185,1534.631945
2016-08-02,id_1001,2452.491092,3404.677476
2016-08-02,id_1003,1916.419934,2370.788136
2016-08-02,id_1004,8424.687306,8882.647361
2016-08-03,id_1000,1401.003123,1446.808327
2016-08-03,id_1001,2417.308783,3307.598144


In [26]:
predictions_array = preds.to_numpy()
prediction_index = preds.index
levels = preds.columns

n_steps, n_levels = predictions_array.shape

df_long = pd.DataFrame({
    #'step':       np.repeat(prediction_index, n_levels),
    'level':      np.tile(levels, n_steps),
    'pred' : predictions_array.ravel()  # o .flatten()
},
index=np.repeat(prediction_index, n_levels))

df_long.head()

,level,pred
2016-08-01,id_1000,1453.312971
2016-08-01,id_1000_lower_bound,1140.542393
2016-08-01,id_1000_upper_bound,1529.475428
2016-08-01,id_1001,2849.347882
2016-08-01,id_1001_lower_bound,2172.050285


In [116]:
# Backtesting
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = regressor, 
                 lags               = 14, 
                 window_features    = RollingFeatures(stats=['mean', 'mean'], window_sizes=[7, 14]),
                 encoding           = "ordinal", 
                 dropna_from_series = False
             )

cv = TimeSeriesFold(
         steps                 = 24,
         initial_train_size    = len(series_dict_train["id_1000"]),
         refit                 = False,
         allow_incomplete_fold = True,
     )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
    forecaster            = forecaster,
    series                = series_dict,
    exog                  = exog_dict,
    cv                    = cv,
    levels                = None,
    metric                = "mean_absolute_error",
    add_aggregated_metric = True,
    n_jobs                ="auto",
    verbose               = False,
    interval              = "bootstrapping",
    n_boot                = 25,
    show_progress         = True,
    suppress_warnings     = True
)

display(metrics_levels)
print("")
display(backtest_predictions)

  0%|          | 0/7 [00:00<?, ?it/s]

,levels,mean_absolute_error
0,id_1000,167.502214
1,id_1001,1103.313887
2,id_1002,NaN
3,id_1003,280.492603
4,id_1004,711.078359
5,average,565.596766
6,weighted_average,535.467442
7,pooling,572.944127


,level,pred,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,...,pred_boot_15,pred_boot_16,pred_boot_17,pred_boot_18,pred_boot_19,pred_boot_20,pred_boot_21,pred_boot_22,pred_boot_23,pred_boot_24
2016-08-01,id_1000,1453.312971,1173.586189,1484.675570,1418.862097,1583.837825,1351.306370,1372.982760,1439.512959,1409.814539,...,1323.631696,1485.943585,1132.281444,983.341389,1398.443783,1483.429494,1474.022549,1409.243375,1470.410114,1470.410114
2016-08-01,id_1001,2849.347882,2690.225531,3315.267219,2119.125183,2462.458516,2708.835260,2462.458516,2690.225531,2577.980086,...,2708.835260,2738.406500,2738.406500,3315.267219,2888.212764,3081.756898,3081.756898,3210.573857,3133.946060,2888.212764
2016-08-01,id_1003,2706.851726,2584.527246,2539.871235,2919.298994,2643.660797,2532.615558,2790.712069,2756.217796,2869.734475,...,2598.919573,2680.117097,2688.405280,2471.135342,2782.541300,2832.232915,2843.289713,2287.494729,2770.632998,2786.290979
2016-08-01,id_1004,7496.555367,7288.915846,5797.501530,7654.538247,7309.451525,7555.799319,7211.963211,7210.689038,7569.243608,...,7746.280512,7309.451525,7203.459456,7206.599391,7607.784914,7566.729479,5797.501530,7288.915846,7218.055198,8310.772954
2016-08-02,id_1000,1440.763196,1434.802833,1500.851630,1448.720409,1479.964783,1440.970575,1402.795567,1531.175270,1404.024956,...,1450.468310,1382.930068,1443.716395,1426.529660,1474.445573,1387.315108,1161.036414,1362.734887,1457.344704,1397.264764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-30,id_1001,1132.535774,486.690637,980.143039,1150.541243,1344.130896,759.371090,1096.960855,1212.352374,622.022276,...,1212.352374,1213.542041,1481.881637,1577.641217,1031.205842,1209.206300,390.432962,755.528966,1031.205842,1470.150355
2016-12-30,id_1003,2089.261345,1898.669067,2026.453904,1996.942433,2118.391255,1823.392510,1772.466935,2109.059187,1687.628235,...,2335.792632,1820.908274,2102.769041,2101.640683,2501.867497,1773.001477,1676.982167,2275.556777,1898.023625,1733.417968
2016-12-31,id_1000,1393.128313,1471.673592,1411.773924,1441.976529,1327.751241,1453.778083,1412.582346,1444.307640,1464.359752,...,1398.314933,1363.110982,1068.748507,1430.000054,1394.973479,1380.985564,1317.879488,1377.544932,1381.832287,1389.415499
2016-12-31,id_1001,1106.034061,947.305218,335.753749,1401.327198,1104.841331,1204.858857,1104.841331,1252.284776,1598.398091,...,1405.346151,785.674870,859.657225,1197.637269,1070.366800,1107.787662,1200.654903,1418.268641,670.153300,365.726196


In [31]:
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures
from sklearn.linear_model import Ridge

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import custom_metric

In [32]:
import joblib

series_dict = joblib.load(
    r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series.joblib"
)

exog_dict = joblib.load(
    r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series_exog.joblib"
)

In [33]:
forecaster = ForecasterRecursiveMultiSeries(
    regressor=LGBMRegressor(
        n_estimators=30, random_state=123, verbose=-1, max_depth=4
    ),
    lags=[1, 7, 14],
    encoding='ordinal',
    dropna_from_series=False,
    transformer_series=None,
    transformer_exog=StandardScaler(),
)

cv = TimeSeriesFold(
            initial_train_size = len(series_dict_train['id_1000']),
            steps              = 24,
            refit              = False
        )

metrics, predictions = backtesting_forecaster_multiseries(
    forecaster        = forecaster,
    series            = series_dict,
    exog              = exog_dict,
    cv                = cv,
    metric            = ['mean_absolute_error', 'mean_absolute_scaled_error'],
    interval          = norm,
    n_boot            = 25,
    n_jobs            = 'auto',
    verbose           = False,
    show_progress     = True,
    suppress_warnings = True
)

  0%|          | 0/7 [00:00<?, ?it/s]

                id_1000  id_1000_loc  id_1000_scale      id_1001  id_1001_loc  \
2016-08-01  1559.691828  1446.935820     171.663434  2934.363292  2738.992298   
2016-08-02  1572.804477  1481.355117      75.017015  3503.747502  2676.919097   
2016-08-03  1537.674947  1453.751766      84.729475  3354.275203  2732.565818   
2016-08-04  1480.694267  1429.975721      98.598613  3537.138899  3099.751220   
2016-08-05  1472.610905  1395.489597      89.077810  3200.844944  2749.721048   
...                 ...          ...            ...          ...          ...   
2016-12-27  1804.827483  1725.149850      86.500613  1345.266413  1165.241384   
2016-12-28  1726.629295  1631.354421      83.605712  1433.532343  1229.613638   
2016-12-29  1622.679349  1544.978673     103.637588  1687.538967  1564.188437   
2016-12-30  1658.431719  1539.075183     140.316399  1797.084928  1545.816578   
2016-12-31  1465.929905  1337.797616     144.814849  1817.367605  1500.298352   

            id_1001_scale  

In [34]:
metrics.to_dict()

{'levels': {0: 'id_1000',
  1: 'id_1001',
  2: 'id_1002',
  3: 'id_1003',
  4: 'id_1004',
  5: 'average',
  6: 'weighted_average',
  7: 'pooling'},
 'mean_absolute_error': {0: 177.94640447766702,
  1: 1451.3480109896332,
  2: nan,
  3: 277.78113362955673,
  4: 993.6769068120083,
  5: 725.1881139772163,
  6: 724.9604804988818,
  7: 724.960480498882},
 'mean_absolute_scaled_error': {0: 0.8178593233613526,
  1: 4.1364664709651064,
  2: nan,
  3: 1.1323827428361022,
  4: 0.8271748048818786,
  5: 1.72847083551111,
  6: 2.0965105153721213,
  7: 1.760615501057647}}

In [35]:
predictions.head(10).to_numpy()

array([[1559.69182787, 1446.93581986,  171.66343358, 2934.36329187,
        2738.99229845,  476.13018745, 3392.60955028, 3285.49512256,
         254.33164031, 7097.05447923, 6901.15599599,  930.8659121 ],
       [1572.80447653, 1481.35511731,   75.01701475, 3503.74750241,
        2676.91909653,  595.80407354, 3118.04939083, 2945.42032254,
         434.03240802, 8301.53364485, 8004.38824145,  905.03703795],
       [1537.67494683, 1453.75176554,   84.7294755 , 3354.2752034 ,
        2732.56581839,  678.25751156, 3118.04939083, 2813.94368549,
         637.92012714, 8466.83628992, 8267.68914986,  792.49132562],
       [1480.69426693, 1429.97572138,   98.59861312, 3537.13889916,
        3099.75121966,  655.2848398 , 2687.48648381, 2302.71838396,
         585.67463419, 8652.97166708, 8620.93678941,  943.37266194],
       [1472.61090534, 1395.4895971 ,   89.0778098 , 3200.84494385,
        2749.72104765,  721.7836647 , 1835.99400007, 1938.17003599,
         444.13125476, 8613.37020561, 8767.7

In [36]:
predictions.columns

Index(['id_1000', 'id_1000_loc', 'id_1000_scale', 'id_1001', 'id_1001_loc',
       'id_1001_scale', 'id_1003', 'id_1003_loc', 'id_1003_scale', 'id_1004',
       'id_1004_loc', 'id_1004_scale'],
      dtype='object')

In [37]:
from skforecast.direct import ForecasterDirect
from sklearn.linear_model import LinearRegression

forecaster = ForecasterDirect(LinearRegression(), lags=3, steps=5)
forecaster.fit(y=pd.Series(np.arange(50)))
last_window = pd.Series(data  = [47, 48, 49], 
                        index = pd.RangeIndex(start=47, stop=50, step=1), name='y')
last_window = pd.Series(data  = [47, 48, 49], 
                        index = pd.RangeIndex(start=47, stop=50, step=1), name='y').to_frame()
results = forecaster.predict(steps=[1, 2, 3, 4], last_window=last_window)

expected = pd.Series(
                data  = np.array([50., 51., 52., 53.]),
                index = pd.RangeIndex(start=50, stop=54, step=1),
                name  = 'pred'
            )

pd.testing.assert_series_equal(results, expected)

In [38]:
last_window

,y
47,47
48,48
49,49


In [44]:
forecaster = ForecasterDirectMultiVariate(
                    regressor          = Ridge(random_state=123),
                    level              = 'l1',
                    lags               = {'l1': 2, 'l2': [1, 3]},
                    steps              = 8,
                    transformer_series = None
                )

cv = TimeSeriesFold(
        initial_train_size = len(series) - 20,
        steps              = 5,
        gap                = 3,
        refit              = False,
        fixed_train_size   = False,
    )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster              = forecaster,
                                            series                  = series,
                                            cv                      = cv,
                                            levels                  = 'l1',
                                            metric                  = 'mean_absolute_error',
                                            add_aggregated_metric   = False,
                                            exog                    = series['l1'].rename('exog_1'),
                                            interval                = norm,
                                            n_boot                  = 150,
                                            random_state            = 123,
                                            use_in_sample_residuals = True,
                                            verbose                 = False
                                        )

  0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
metrics_levels

,levels,mean_absolute_error
0,l1,0.117919


In [46]:
backtest_predictions.to_numpy()

array([[0.55880533, 0.55863664, 0.11294946],
       [0.46285725, 0.46532544, 0.11662733],
       [0.35358667, 0.34633493, 0.10823561],
       [0.44404948, 0.43289758, 0.10734709],
       [0.64659616, 0.65154034, 0.10756972],
       [0.70306475, 0.70289606, 0.11294946],
       [0.48677757, 0.48924576, 0.11662733],
       [0.49848981, 0.49123807, 0.10823561],
       [0.31544893, 0.30429703, 0.10734709],
       [0.4450306 , 0.44997478, 0.10756972],
       [0.50164877, 0.50148008, 0.11294946],
       [0.62883248, 0.63130067, 0.11662733],
       [0.33387601, 0.32662427, 0.10823561],
       [0.45961408, 0.44846217, 0.10734709],
       [0.63726975, 0.64221393, 0.10756972],
       [0.54013414, 0.53996545, 0.11294946],
       [0.52550978, 0.52797797, 0.11662733]])

In [48]:
backtest_predictions.columns

Index(['l1', 'l1_loc', 'l1_scale'], dtype='object')

In [ ]:
expected = pd.DataFrame(
            data = np.array([[0.6211422 , 0.73248662],
                            [0.42283865, 0.45391191],
                            [0.49351412, 0.661996  ],
                            [0.71846231, 0.58080355],
                            [0.55089719, 0.59780378],
                            [0.39224631, 0.40316854],
                            [0.46827996, 0.51400857],
                            [0.67707084, 0.42834292],
                            [0.45119292, 0.4503941 ],
                            [0.61998977, 0.65552498]]),
            index   = pd.RangeIndex(start=50, stop=60, step=1),
            columns = ['1', '2']
        )
expected

,1,2
50,0.621142,0.732487
51,0.422839,0.453912
52,0.493514,0.661996
53,0.718462,0.580804
54,0.550897,0.597804
55,0.392246,0.403169
56,0.468280,0.514009
57,0.677071,0.428343
58,0.451193,0.450394
59,0.619990,0.655525
